In [ ]:
from happytransformer import HappyTextToText
from datasets import load_dataset
from happytransformer import TTSettings
import csv
from happytransformer import TTTrainArgs

In [ ]:
happy_tt1 = HappyTextToText("T5", "t5-base")
happy_tt2 = HappyTextToText("T5", "addy88/t5-grammar-correction")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
train_dataset = load_dataset("jfleg", split="validation[:]")
eval_dataset = load_dataset("jfleg", split="test[:]")

In [ ]:
for i in range(0, 2):
  print(f"{train_dataset['corrections'][i][0]}: {train_dataset['corrections'][i]}")
  print("--------------------------------------------------------")

So I think we would not be alive if our ancestors did not develop sciences and technologies . : ['So I think we would not be alive if our ancestors did not develop sciences and technologies . ', 'So I think we could not live if older people did not develop science and technologies . ', 'So I think we can not live if old people could not find science and technologies and they did not develop . ', 'So I think we can not live if old people can not find the science and technology that has not been developed . ']
--------------------------------------------------------
Not for use with a car . : ['Not for use with a car . ', 'Do not use in the car . ', 'Car not for use . ', 'Can not use the car . ']
--------------------------------------------------------


In [ ]:
replacements = [
  (" .", "."),
  (" ,", ","),
  (" '", "'"),
  (" ?", "?"),
  (" !", "!"),
  (" :", "!"),
  (" ;", "!"),
  (" n't", "n't"),
  (" v", "n't"),
  ("2 0 0 6", "2006"),
  ("5 5", "55"),
  ("4 0 0", "400"),
  ("1 7-5 0", "1750"),
  ("2 0 %", "20%"),
  ("5 0", "50"),
  ("1 2", "12"),
  ("1 0", "10"),
  ('" ballast water', '"ballast water')
]

def remove_excess_spaces(text):
  for rep in replacements:
    text = text.replace(rep[0], rep[1])

  return text

In [ ]:
def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
     	    # Adding the task's prefix to input
            input_text = "grammar: " + case["sentence"]
            for correction in case["corrections"]:
                # a few of the cases contain blank strings.
                if input_text and correction:
                    input_text = remove_excess_spaces(input_text)
                    correction = remove_excess_spaces(correction)
                    writter.writerow([input_text, correction])

In [ ]:
generate_csv("train.csv", train_dataset)
generate_csv("eval.csv", eval_dataset)

In [ ]:
args = TTTrainArgs(batch_size=8)
happy_tt1.train("train.csv", args=args)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2714 [00:00<?, ? examples/s]

Map:   0%|          | 0/302 [00:00<?, ? examples/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
1,1.575200,1.438123
34,0.883800,0.701032
68,0.803700,0.609520
102,0.717600,0.577185
136,0.674200,0.566056
170,0.674900,0.554227
204,0.678800,0.543009
238,0.649300,0.535889
272,0.616800,0.532202
306,0.592100,0.529834


In [ ]:
before_loss = happy_tt1.eval("eval.csv")
print("After loss: ", before_loss.loss)

Map:   0%|          | 0/2714 [00:00<?, ? examples/s]

Map:   0%|          | 0/302 [00:00<?, ? examples/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
1,0.503300,0.350423
34,0.389000,0.359672
68,0.438500,0.350615
102,0.443500,0.348483
136,0.434200,0.351147
170,0.427500,0.353540
204,0.447900,0.351648
238,0.427500,0.348890
272,0.407500,0.349163
306,0.403500,0.348627


In [ ]:
beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=20)

In [ ]:
example_1 = "grammar: This sentences, has bads grammar and spelling!"
result_1 = happy_tt1.generate_text(example_1, args=beam_settings)
print(result_1.text)

This sentences, has bad grammar and spelling!


In [ ]:
happy_tt1.save("model/")